In [1]:
import requests
import xml.etree.ElementTree as ET
from html.parser import HTMLParser

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [2]:
requests.get?

Signature: requests.get(url, params=None, **kwargs)
Docstring:
Sends a GET request.

:param url: URL for the new :class:`Request` object.
:param params: (optional) Dictionary, list of tuples or bytes to send
    in the query string for the :class:`Request`.
:param \*\*kwargs: Optional arguments that ``request`` takes.
:return: :class:`Response <Response>` object
:rtype: requests.Response
File:      ~/micromamba/lib/python3.10/site-packages/requests/api.py
Type:      function

In [25]:

def download_html(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None


class MyHTMLParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.capture_data = False
        self.target_class = ""
        self.captured_data = []
        self.file_ids = []

    def handle_starttag(self, tag, attrs):
        if tag == "tr":
            for attr in attrs:
                if attr[0] == self.target_class:
                    # print(attr[0], attr[1])
                    self.file_ids.append(attr[1])
                    self.capture_data = True

    def handle_endtag(self, tag):
        if tag == "tr" and self.capture_data:
            self.capture_data = False

    def handle_data(self, data):
        if self.capture_data:
            self.captured_data.append(data)

    def extract_span_by_class(self, html, class_value):
        self.target_class = class_value
        self.captured_data = []
        self.feed(html)
        # return self.captured_data
        return self.file_ids


In [6]:

url = "https://www.example.com"
url = "https://filesender.aarnet.edu.au/?s=download&token=d6be9395-9fc6-427f-b565-289cb32a16e2"
url = "https://filesender.aarnet.edu.au/?s=download&token=205a5f13-6333-4f56-b126-742ebf09691e"

html_content = download_html(url)

In [7]:
html_content

'<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml" lang="en-au" xml:lang="en-au">\n    <head>\n        <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\n        <meta charset="utf-8">\n        <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\n        <meta http-equiv="x-ua-compatible" content="ie=edge">\n        <title>AARNet FileSender</title>\n        <link type="text/css" rel="stylesheet" href="/lib/jquery-ui/jquery-ui.min.css" />\n        <link type="text/css" rel="stylesheet" href="/lib/font-awesome/css/font-awesome.min.css" />\n        <link type="text/css" rel="stylesheet" href="/lib/bootstrap/dist/css/bootstrap.min.css" />\n        <link type="text/css" rel="stylesheet" href="/lib/flag-icons/css/flag-icons.min.css" />\n        <link type="text/css" rel="stylesheet" href="/css/default.css" />\n        <link type="text/css" rel="stylesheet" href="/css/new-ui/styles.css" />\n        <link type="text/css" rel="sty

In [27]:

parser = MyHTMLParser()
directlinks = parser.extract_span_by_class(html_content, "directlink")
directlinks = [x.split("Direct Link: ")[1].strip() for x in directlinks]
token = url.split("&token=")[1]
fileids = [x.split("&files_ids=")[1] for x in directlinks]


In [29]:
fileids = parser.extract_span_by_class(html_content, "data-id")

In [30]:
baseurl = "https://filesender.aarnet.edu.au/download.php"
token="TOKEN"
directlinks = [ f"{baseurl}?token={token}&files_ids={x}" for x in fileids ]


In [31]:
directlinks

['https://filesender.aarnet.edu.au/download.php?token=TOKEN&files_ids=22649701',
 'https://filesender.aarnet.edu.au/download.php?token=TOKEN&files_ids=22649698',
 'https://filesender.aarnet.edu.au/download.php?token=TOKEN&files_ids=22649704',
 'https://filesender.aarnet.edu.au/download.php?token=TOKEN&files_ids=22649701',
 'https://filesender.aarnet.edu.au/download.php?token=TOKEN&files_ids=22649698',
 'https://filesender.aarnet.edu.au/download.php?token=TOKEN&files_ids=22649704']

In [ ]:




def parse_url(u):
    prefix = "https://filesender.aarnet.edu.au/?s=download"
    if u.startswith("prefix") == False:
        return False, "wrong prefix"
    bits = u[len(prefix)+1:].split("&")
    token = ""
    for b_ in bits:
        if b_.startswith("token_"):
            token = b_.split("=")[1]
    return True, token



In [ ]:
u="https://filesender.aarnet.edu.au/download.php?token=d6be9395-9fc6-427f-b565-289cb32a16e2&files_ids=22322589%2C22322355%2C22322460%2C22322286%2C22322646%2C22322397%2C22322496%2C22322271%2C22322595%2C22322385%2C22322676%2C22322358%2C22322547%2C22322253%2C22322661%2C22322340%2C22322640%2C22322313%2C22322685%2C22322361%2C22322619%2C22322301%2C22322631%2C22322319%2C22322190%2C22322136%2C22322127%2C22322124%2C22322265%2C22322196%2C22322217%2C22322163%2C22322229%2C22322157%2C22322205%2C22322148%2C22322199%2C22322145%2C22322247%2C22322184%2C22322232%2C22322166%2C22322244%2C22322175%2C22322322%2C22322250%2C22322220%2C22322169%2C22322379%2C22322316%2C22322292%2C22322226%2C22322574%2C22322406%2C22322505%2C22322388%2C22322562%2C22322400%2C22322610%2C22322454%2C22322613%2C22322481%2C22322529%2C22322442%2C22322553%2C22322448%2C22322541%2C22322472%2C22322568%2C22322520%2C22322478%2C22322433%2C22322469%2C22322331%2C22322304%2C22322238%2C22322367%2C22322259%2C22322376%2C22322295%2C22322268%2C22322193%2C22322283%2C22322208%2C22322370%2C22322274%2C22322256%2C22322181%2C22322334%2C22322223%2C22322463%2C22322307%2C22322415%2C22322280%2C22322364%2C22322262%2C22322424%2C22322277%2C22322142%2C22322130%2C22322298%2C22322211%2C22322235%2C22322172%2C22322325%2C22322241%2C22322289%2C22322214%2C22322160%2C22322133%2C22322178%2C22322154%2C22322373%2C22322337%2C22322202%2C22322187%2C22322151%2C22322139%2C22322592%2C22322538%2C22322487%2C22322403%2C22322571%2C22322475%2C22322604%2C22322502%2C22322625%2C22322577%2C22322682%2C22322532%2C22322679%2C22322535%2C22322583%2C22322421%2C22322664%2C22322511%2C22322673%2C22322526%2C22322655%2C22322445%2C22322607%2C22322412%2C22322634%2C22322439%2C22322499%2C22322310%2C22322517%2C22322328%2C22322556%2C22322346%2C22322550%2C22322352%2C22322598%2C22322394%2C22322667%2C22322484%2C22322616%2C22322409%2C22322628%2C22322430%2C22322652%2C22322457%2C22322565%2C22322343%2C22322586%2C22322391%2C22322580%2C22322382%2C22322559%2C22322349%2C22322643%2C22322466%2C22322637%2C22322436%2C22322601%2C22322418%2C22322691%2C22322508%2C22322697%2C22322544%2C22322649%2C22322451%2C22322658%2C22322490%2C22322688%2C22322523%2C22322694%2C22322514%2C22322622%2C22322427%2C22322670%2C22322493%2C22322121%2C22322700%2C22322703&archive_format=zip&transaction_id=0fa22c7f-d336-49d7-9d28-5738601762d4"

u="https://filesender.aarnet.edu.au/?s=download&token=d6be9395-9fc6-427f-b565-289cb32a16e2"

In [ ]:
prefix = "https://filesender.aarnet.edu.au/?s=download"
if u.startswith("prefix") == False:
    print("wrong prefix")
bits = u[len(prefix)+1:].split("&")
bits